In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
run_ts = spark.sql("SELECT current_timestamp()").collect()[0][0]

In [0]:
def log_and_print(layer, table_name, metric_type, metric_value):
    # Print to notebook
    print(f"[{layer.upper()}] {table_name} | {metric_type}: {metric_value}")

    # Persist to Delta log table
    spark.createDataFrame(
        [(run_ts, layer, table_name, metric_type, metric_value)],
        ["run_timestamp", "layer", "table_name", "metric_type", "metric_value"]
    ).write.mode("append").saveAsTable("pipeline_data_quality_log")


In [0]:
bronze_tables = [
    "bronze_sales_transactions",
    "bronze_product_master",
    "bronze_store_region"
]

print("\n🔹 BRONZE LAYER COUNTS")
for table in bronze_tables:
    count = spark.table(table).count()
    log_and_print("bronze", table, "record_count", count)


In [0]:
silver_tables = [
    "silver_sales_transactions",
    "silver_product_master",
    "silver_store_region"
]

print("\n🔹 SILVER LAYER COUNTS")
for table in silver_tables:
    count = spark.table(table).count()
    log_and_print("silver", table, "record_count", count)



🔹 SILVER LAYER COUNTS


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-8275285343330475>, line 10
      8 for table in silver_tables:
      9     count = spark.table(table).count()
---> 10     log_and_print("silver", table, "record_count", count)

NameError: name 'log_and_print' is not defined

In [0]:
gold_tables = [
    "gold_daily_sales_summary",
    "gold_monthly_revenue_by_region",
    "gold_product_performance"
]

print("\n🔹 GOLD LAYER COUNTS")
for table in gold_tables:
    count = spark.table(table).count()
    log_and_print("gold", table, "record_count", count)


In [0]:
rejected_df = (
    spark.table("silver_sales_quarantine")
    .groupBy("rejection_reason")
    .count()
)

for row in rejected_df.collect():
    reason = row["rejection_reason"]
    count = row["count"]

    log_and_print(
        "silver",
        "silver_sales_quarantine",
        f"rejected_{reason}",
        count
    )
